In [1]:
!pip install ollama

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
# import os
# os.environ["OLLAMA_HOST"] = "https://541d-34-16-156-236.ngrok-free.app"

In [3]:
!ollama pull codellama:7b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 3a43f93b78ec...   0% ▕                ▏    0 B/3.8 GB                  pulling manifest 
pulling 3a43f93b78ec...   0% ▕                ▏    0 B/3.8 GB                  pulling manifest 
pulling 3a43f93b78ec...   0% ▕                ▏    0 B/3.8 GB                  pulling manifest 
pulling 3a43f93b78ec...   0% ▕                ▏    0 B/3.8 GB                  pulling manifest 
pulling 3a43f93b78ec...   0% ▕                ▏    0 B/3.8 GB                  pulling manifest 
pulling 3a43f93b78ec...   0% ▕                ▏    0 B/3.8 GB                  pulling manifest 
pulling 3a43f93b78ec...   0% ▕                ▏ 3.0 MB/3.8 GB                  pulling manifest 

In [5]:
!ollama pull codellama:7b
!ollama pull gemma3:4b
#!ollama pull qwen2.5:3b
# !ollama pull qwen2.5:3b

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 3a43f93b78ec... 100% ▕████████████████▏ 3.8 GB                         
pulling 8c17c2ebb0ea... 100% ▕████████████████▏ 7.0 KB                         
pulling 590d74a5569b... 100% ▕████████████████▏ 4.8 KB                         
pulling 2e0493f67d0c... 100% ▕████████████████▏   59 B                         
pulling 7f6a57943a88... 100% ▕████████████████▏  120 B                         
pulling 316526ac7323... 100% ▕████████████████▏  529 B                         
verifying sha256 digest 
writing manifest 
success 
pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠋ pulling manifest 
pulling aeda25e63ebd...   0% ▕                ▏    0 B/3.3 GB                  pulling man

In [20]:
!ollama pull qwen2.5:3b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 5ee4f07cdb9b...   0% ▕                ▏    0 B/1.9 GB                  pulling manifest 
pulling 5ee4f07cdb9b...   0% ▕                ▏    0 B/1.9 GB                  pulling manifest 
pulling 5ee4f07cdb9b...   0% ▕                ▏    0 B/1.9 GB                  pulling manifest 
pulling 5ee4f07cdb9b...   0% ▕                ▏    0 B/1.9 GB                  pulling manifest 
pulling 5ee4f07cdb9b...   0% ▕                ▏    0 B/1.9 GB                  pulling manifest 
pulling 5ee4f07cdb9b...   0% ▕                ▏    0 B/1.9 GB                  pulling manifest 
pulling 5ee4f07cdb9b...   0% ▕                ▏  65 KB/1.9 GB                  pulling manifest 
pulling 5ee4f07cdb

In [21]:
!ollama list

NAME             ID              SIZE      MODIFIED               
qwen2.5:3b       357c53fb659c    1.9 GB    Less than a second ago    
gemma3:4b        a2af6cc3eb7f    3.3 GB    4 minutes ago             
codellama:7b     8fdf8f752f6e    3.8 GB    6 minutes ago             
llama3:latest    365c0bd3c000    4.7 GB    5 days ago                


In [7]:
import ollama
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os
import pandas as pd

In [46]:
def read_file(filename):
    file = open(filename,'r')
    str = file.read()
    return str

In [53]:
def get_custom_prompt(question, sql_file):
    def read_file(file_path):
        with open(file_path, 'r') as f:
            return f.read()

    prompt = f"""
### Task:
Translate the following natural language question into a SQL query using the provided PostgreSQL database schema.

### Guidelines:
- Carefully analyze **each word** of the question and the schema to generate the correct query.
- Always use **table aliases** to ensure clarity and avoid ambiguity.  
  Example: `SELECT t1.col1, t2.col1 FROM table1 t1 JOIN table2 t2 ON t1.id = t2.id`
- When calculating ratios, **cast the numerator to float** to maintain precision.
- Respond with only SQL Query. No additional Text or Explaination.

### Question:
Generate a SQL query that answers: `{question}`

The schema of the target database is as follows:

{read_file(sql_file)}

### Output:
```sql
"""
    return prompt


In [64]:
def get_rate_prompt(schema, question, answer):
    prompt = f"""
### Task
You are provided with:
- A PostgreSQL database schema
- A natural language question about the data
- A SQL query that attempts to answer the question

Your task is to **evaluate the SQL query** based on its correctness, structure, and adherence to best practices.

### Evaluation Criteria
- Read and analyze the **question** and **schema** thoroughly.
- Ensure the SQL logic **correctly answers the question**.
- Verify that **appropriate table aliases** are used for clarity.
- If any **ratios or percentage calculations** are involved, ensure the **numerator is explicitly cast to FLOAT or DECIMAL** for accuracy.

### Input
- Question: {question}
- SQL Query: {answer}
- Schema:
{read_file(schema)}

### Output
Return a **single integer rating** from **1 to 10**, where:
- 10 = perfect and optimized query
- 1 = completely incorrect or unrelated

**Output only the integer. No comments, no explanation, no formatting.**
"""
    return prompt


def get_ratings(schema,question,answer):
    response = llm_gemini.invoke(
        get_rate_prompt(
            schema = schema,
            question = question,
            answer = answer,
            )
    )
    
    return response.content


In [65]:
load_dotenv()

llm_gemini = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash",
    api_key = os.getenv("GEMINI_API_KEY")
)


In [66]:
questions = [
    "List all drugs in the 'Painkillers' category with a stock below 100 units, along with their supplier names and the date when their inventory was last updated. Order results by oldest inventory update first.",
    "Identify the top 5 products by total sales amount in the last quarter of 2024, including their category names and the store locations where they were sold. Order the results by total sales amount in descending order.",
    "Find the top 3 products with the highest average rating from user reviews, including their category names and total sales amounts. Only consider products that have been ordered at least 10 times. Order the results by average rating in descending order.",
    "List all agents who have facilitated sales of properties with a total sale price exceeding $500,000 in the last year. Include their full names, the number of properties sold, and the total commission earned. Order the results by total commission in descending order.",
    "Identify the top 5 products by total sales amount in the last quarter of 2024, including the name of the employee who made the sale and the total quantity sold for each product. Order the results by total sales amount in descending order.",    
]

responses = []
ratings = []

In [67]:
!ollama pull llama3.2:3b
!ollama list

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


NAME             ID              SIZE      MODIFIED               
llama3.2:3b      a80c4f17acd5    2.0 GB    Less than a second ago    
qwen2.5:3b       357c53fb659c    1.9 GB    31 minutes ago            
gemma3:4b        a2af6cc3eb7f    3.3 GB    35 minutes ago            
codellama:7b     8fdf8f752f6e    3.8 GB    37 minutes ago            
llama3:latest    365c0bd3c000    4.7 GB    5 days ago                


In [68]:
results = []

In [69]:
# model_list = ['qwen2.5-coder:7b','gemma3:4b','deepseek-coder:6.7b']
model_list = ['qwen2.5:3b','gemma3:4b','llama3.2:3b','codellama:7b','llama3:latest']
# model_list = ['qwen2.5:3b']

for model in model_list:
    for i in range(5):
        FILENAME = f"table_list{i+1}_exp.sql"

        response = ollama.chat(model=model, messages=[
        {
            'role': 'user',
            # 'content': get_prompt("What are the names and prices of all drugs in the 'Antibiotics' category?")
            'content': get_custom_prompt(
                question = questions[i],
                sql_file = FILENAME
            )
            # 'content': get_prompt("Which pharmacies have sold more than 100 units of drugs from the 'Painkillers' category in the last 30 days, and what is the total revenue generated from those sales, including the names of the employees who made the sales?")
        },
        ])

        res = response['message']['content']

        responses.append(res)

        rating = get_ratings(
            schema = f"table_list{i+1}_exp.sql",
            question = questions[i],
            answer = res,
        )

        ratings.append(rating)

        dict = {
            'model_name' : model,
            'schema' : read_file(FILENAME),
            'question' : questions[i],
            'sql_query' : res,
            'ratings' : rating,
        }

        results.append(dict)

In [70]:
# dict = {
#     model_name = model.
#     sql_query = query,
#     ratings = ratings
# }

In [71]:
results

[{'model_name': 'qwen2.5:3b',
  'schema': "    \n    CREATE TABLE supplier (\n    supplier_id INT PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each supplier, auto-incremented\n    supplier_name VARCHAR(100) NOT NULL, -- Name of the supplier company or entity\n    contact_info VARCHAR(100), -- Contact details (e.g., phone, email) of the supplier\n    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP -- Date and time when the supplier record was created\n);\n\nCREATE TABLE drug_category (\n    category_id INT PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each drug category, auto-incremented\n    category_name VARCHAR(50) NOT NULL -- Name of the drug category (e.g., Antibiotics, Painkillers)\n);\n\nCREATE TABLE drug (\n    drug_id INT PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each drug, auto-incremented\n    category_id INT, -- ID of the category this drug belongs to\n    drug_name VARCHAR(100) NOT NULL, -- Name of the drug (e.g., Amoxicillin)\n    manufacturer VARCHAR(100), -- Name of the

In [72]:
import pandas as pd

df = pd.DataFrame(results)

df.to_csv('results-3.csv')


In [73]:
df

,model_name,schema,question,sql_query,ratings
0,qwen2.5:3b,\n CREATE TABLE supplier (\n supplie...,List all drugs in the 'Painkillers' category w...,"```sql\nSELECT d.drug_name, s.supplier_name, i...",9
1,qwen2.5:3b,CREATE TABLE store (\n store_id INT PRIMARY...,Identify the top 5 products by total sales amo...,```sql\nWITH sales_total AS (\n SELECT \n ...,5
2,qwen2.5:3b,CREATE TABLE user (\n user_id INT PRIMARY K...,Find the top 3 products with the highest avera...,```sql\nWITH product_ratings AS (\n SELECT ...,4
3,qwen2.5:3b,CREATE TABLE agent (\n agent_id INT PRIMARY...,List all agents who have facilitated sales of ...,```sql\nSELECT \n a.first_name AS AgentFirs...,8
4,qwen2.5:3b,CREATE TABLE supplier (\n supplier_id INT P...,Identify the top 5 products by total sales amo...,"```sql\nSELECT \n p.product_name,\n e.fi...",8
5,gemma3:4b,\n CREATE TABLE supplier (\n supplie...,List all drugs in the 'Painkillers' category w...,"```sql\nSELECT\n d.drug_name,\n s.supplier_n...",9
6,gemma3:4b,CREATE TABLE store (\n store_id INT PRIMARY...,Identify the top 5 products by total sales amo...,"```sql\nSELECT\n p.product_name,\n c.categor...",6
7,gemma3:4b,CREATE TABLE user (\n user_id INT PRIMARY K...,Find the top 3 products with the highest avera...,"```sql\nSELECT\n p.product_name,\n c.categor...",7
8,gemma3:4b,CREATE TABLE agent (\n agent_id INT PRIMARY...,List all agents who have facilitated sales of ...,```sql\nSELECT\n a.first_name || ' ' || a.las...,8
9,gemma3:4b,CREATE TABLE supplier (\n supplier_id INT P...,Identify the top 5 products by total sales amo...,"```sql\nSELECT\n p.product_name,\n e.first_n...",6
